##1. Download Data

In [2]:
!ls -l
from google.colab import drive
drive.mount('/content/gdrive')

total 4
drwxr-xr-x 1 root root 4096 Jan  8 17:15 sample_data
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

In [3]:
!curl -X GET https://raw.githubusercontent.com/tensorflow/nmt/master/nmt/scripts/download_iwslt15.sh -o download_iwslt15.sh
!chmod +x download_iwslt15.sh && ./download_iwslt15.sh nmt_data

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1188  100  1188    0     0   7200      0 --:--:-- --:--:-- --:--:--  7200
mkdir: created directory 'nmt_data'
Download training dataset train.en and train.vi.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 12.9M  100 12.9M    0     0   720k      0  0:00:18  0:00:18 --:--:--  735k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 17.2M  100 17.2M    0     0   691k      0  0:00:25  0:00:25 --:--:--  532k
Download dev dataset tst2012.en and tst2012.vi.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Spee

In [11]:
# Upload the file to Drive. See:
#
# https://developers.google.com/drive/v3/reference/files/create
# https://developers.google.com/drive/v3/web/manage-uploads
from googleapiclient.http import MediaFileUpload

file_metadata = {
  'name': 'models.zip',
  'mimeType': 'text/plain'
}
media = MediaFileUpload('download_iwslt15.sh', 
                        mimetype='application/octet-stream',
                        resumable=True)
created = drive_service.files().create(body=file_metadata,
                                       media_body=media,
                                       fields='id').execute()
print('File ID: {}'.format(created.get('id')))

File ID: 15DUb94RPbIOsaEzwEwhJXk2X3-kFkr9P


In [4]:
!ls -l

total 16
-rwxr-xr-x 1 root root 1188 Jan 21 02:29 download_iwslt15.sh
drwx------ 3 root root 4096 Jan 21 02:28 gdrive
drwxr-xr-x 2 root root 4096 Jan 21 02:30 nmt_data
drwxr-xr-x 1 root root 4096 Jan  8 17:15 sample_data


## 2. Clone code

In [12]:
!git clone https://github.com/tensorflow/nmt.git

Cloning into 'nmt'...
remote: Enumerating objects: 1247, done.
remote: Total 1247 (delta 0), reused 0 (delta 0), pack-reused 1247
Receiving objects: 100% (1247/1247), 1.22 MiB | 8.85 MiB/s, done.
Resolving deltas: 100% (892/892), done.


In [13]:
!ls -l nmt/nmt

total 236
-rw-r--r-- 1 root root  7387 Jan 21 02:37 attention_model.py
drwxr-xr-x 3 root root  4096 Jan 21 02:37 g3doc
-rw-r--r-- 1 root root 12252 Jan 21 02:37 gnmt_model.py
-rw-r--r-- 1 root root  8895 Jan 21 02:37 inference.py
-rw-r--r-- 1 root root  6490 Jan 21 02:37 inference_test.py
-rw-r--r-- 1 root root     0 Jan 21 02:37 __init__.py
-rw-r--r-- 1 root root 24395 Jan 21 02:37 model_helper.py
-rw-r--r-- 1 root root 33588 Jan 21 02:37 model.py
-rw-r--r-- 1 root root 48465 Jan 21 02:37 model_test.py
-rw-r--r-- 1 root root 29123 Jan 21 02:37 nmt.py
-rw-r--r-- 1 root root  3404 Jan 21 02:37 nmt_test.py
drwxr-xr-x 2 root root  4096 Jan 21 02:37 scripts
drwxr-xr-x 2 root root  4096 Jan 21 02:37 standard_hparams
drwxr-xr-x 2 root root  4096 Jan 21 02:37 testdata
-rw-r--r-- 1 root root 28948 Jan 21 02:37 train.py
drwxr-xr-x 2 root root  4096 Jan 21 02:37 utils


## 3. Tensor Board

### Download

In [14]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!rm -rf ngrok
!unzip ngrok-stable-linux-amd64.zip

--2019-01-21 02:37:28--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.204.22.7, 34.206.9.96, 34.206.253.53, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.204.22.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]   5.11M  20.3MB/s    in 0.3s    

2019-01-21 02:37:29 (20.3 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


## 4. Training

### 4.1 Basic

In [7]:
tsboard_cmd = 'tensorboard --logdir nmt_model --host 0.0.0.0 --port 6006 &'
print("Log dir", tsboard_cmd)
get_ipython().system_raw(tsboard_cmd)
get_ipython().system_raw('./ngrok http 6006 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

Log dir tensorboard --logdir nmt_model --host 0.0.0.0 --port 6006 &
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/lib/python3.6/json/__init__.py", line 106, in <module>
    from .decoder import JSONDecoder, JSONDecodeError
  File "/usr/lib/python3.6/json/decoder.py", line 53, in <module>
    STRINGCHUNK = re.compile(r'(.*?)(["\\\x00-\x1f])', FLAGS)
  File "/usr/lib/python3.6/re.py", line 233, in compile
    return _compile(pattern, flags)
  File "/usr/lib/python3.6/re.py", line 301, in _compile
    p = sre_compile.compile(pattern, flags)
  File "/usr/lib/python3.6/sre_compile.py", line 566, in compile
    code = _code(p, flags)
  File "/usr/lib/python3.6/sre_compile.py", line 551, in _code
    _compile(code, p.data, flags)
  File "/usr/lib/python3.6/sre_compile.py", line 146, in _compile
    _compile(code, p, (flags | add_flags) & ~del_flags)
  File "/usr/lib/python3.6/sre_compile.py", line 72, in _compile
    if (flags & SRE_FLAG_IGNORECASE and

KeyboardInterrupt: ignored

In [0]:
!rm -rf nmt_model 
!mkdir nmt_model
!python -m nmt.nmt.nmt \
    --src=vi --tgt=en \
    --vocab_prefix=nmt_data/vocab  \
    --train_prefix=nmt_data/train \
    --dev_prefix=nmt_data/tst2012  \
    --test_prefix=nmt_data/tst2013 \
    --out_dir=nmt_model \
    --num_train_steps=12000 \
    --steps_per_stats=100 \
    --num_layers=4 \
    --num_units=128 \
    --dropout=0.2 \
    --metrics=bleu

In [0]:
!ls -l nmt_model

ls: cannot access 'nmt_model': No such file or directory


### 4.2 Attention

In [15]:
# Clean old model
!rm -rf gdrive/nmt_attention_model
!mkdir gdrive/nmt_attention_model

mkdir: cannot create directory ‘gdrive/nmt_attention_model’: Operation not supported


In [16]:
tsboard_cmd = 'tensorboard --logdir nmt_attention_model --host 0.0.0.0 --port 6006 &'
print("Log dir", tsboard_cmd)
get_ipython().system_raw(tsboard_cmd)
get_ipython().system_raw('./ngrok http 6006 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

Log dir tensorboard --logdir nmt_attention_model --host 0.0.0.0 --port 6006 &
http://0c7abc4e.ngrok.io


In [11]:
!curl -X GET https://raw.githubusercontent.com/tensorflow/nmt/master/nmt/standard_hparams/iwslt15.json -o params.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   823  100   823    0     0   7550      0 --:--:-- --:--:-- --:--:--  7550


In [20]:
!python -m nmt.nmt.nmt \
    --attention=scaled_luong \
    --src=vi --tgt=en \
    --vocab_prefix=nmt_data/vocab  \
    --train_prefix=nmt_data/train \
    --dev_prefix=nmt_data/tst2012  \
    --test_prefix=nmt_data/tst2013 \
    --out_dir=gdrive/My\ Drive/nmt_attention_model \
    --num_train_steps=12000 \
    --steps_per_stats=100 \
    --infer_mode=beam_search\
    --beam_width=10\
    --num_layers=2 \
    --num_units=512 \
    --dropout=0.2 \
    --metrics=bleu \
    --encoder_type=bi\
    --decay_scheme=luong234
    

# Job id 0
2019-01-21 05:08:44.199210: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:964] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-01-21 05:08:44.199958: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1432] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
totalMemory: 11.17GiB freeMemory: 11.10GiB
2019-01-21 05:08:44.200029: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1511] Adding visible gpu devices: 0
2019-01-21 05:08:44.725430: I tensorflow/core/common_runtime/gpu/gpu_device.cc:982] Device interconnect StreamExecutor with strength 1 edge matrix:
2019-01-21 05:08:44.725520: I tensorflow/core/common_runtime/gpu/gpu_device.cc:988]      0 
2019-01-21 05:08:44.725543: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1001] 0:   N 
2019-01-21 05:08:44.725853: W tensorflow/core/common_runtime/gpu/gpu_bfc_a

In [13]:
!rm -f models.zip
!zip -r9 models.zip nmt_attention_model


  adding: nmt_attention_model/ (stored 0%)
  adding: nmt_attention_model/translate.ckpt-8000.index (deflated 44%)
  adding: nmt_attention_model/translate.ckpt-12000.index (deflated 44%)
  adding: nmt_attention_model/translate.ckpt-8000.data-00000-of-00001 (deflated 8%)
  adding: nmt_attention_model/translate.ckpt-9000.meta (deflated 93%)
  adding: nmt_attention_model/translate.ckpt-10000.data-00000-of-00001 (deflated 7%)
  adding: nmt_attention_model/checkpoint (deflated 75%)
  adding: nmt_attention_model/translate.ckpt-11000.index (deflated 44%)
  adding: nmt_attention_model/translate.ckpt-12000.data-00000-of-00001 (deflated 7%)
  adding: nmt_attention_model/output_dev (deflated 67%)
  adding: nmt_attention_model/best_bleu/ (stored 0%)
  adding: nmt_attention_model/best_bleu/translate.ckpt-7000.meta (deflated 92%)
  adding: nmt_attention_model/best_bleu/translate.ckpt-7000.index (deflated 44%)
  adding: nmt_attention_model/best_bleu/translate.ckpt-6000.index (deflated 44%)
  adding: n